In [1]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [3]:
df = pd.read_csv("shoes_data_20k.csv")

In [4]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR]


# Finding outlier 

In [5]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return round(low_limit), round(up_limit)

In [6]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000,3.111,4.226,1.000,1.000,2.000,4.000,200.000
order_num_total_ever_offline,19945.000,1.914,2.063,1.000,1.000,1.000,2.000,109.000
customer_value_total_ever_offline,19945.000,253.923,301.533,10.000,99.990,179.980,319.970,18119.140
customer_value_total_ever_online,19945.000,497.322,832.602,12.990,149.980,286.460,578.440,45220.130


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [13]:
for col in df.columns:
    if df[col].dtype=="float":
        replace_with_thresholds(df, col) #In this code, i replaced thresholds for the numeric variables

In [14]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000,3.092,3.810,1.000,1.000,2.000,4.000,48.000
order_num_total_ever_offline,19945.000,1.886,1.435,1.000,1.000,1.000,2.000,16.000
customer_value_total_ever_offline,19945.000,251.921,251.024,10.000,99.990,179.980,319.970,3020.000
customer_value_total_ever_online,19945.000,489.706,632.610,12.990,149.980,286.460,578.440,7800.000
omni_channel,19945.000,4.978,4.116,2.000,3.000,4.000,6.000,52.000


#### Omni_channel is sum of the online and offline order

In [10]:
df["omni_channel"]  = df["order_num_total_ever_offline"] + df["order_num_total_ever_online"]

#### I changed the type of the variables that is date to datetime

In [11]:
(df['first_order_date'],df['last_order_date'],df['last_order_date_online'],df['last_order_date_offline'])=[pd.to_datetime(df[col]) for col in df.columns if "date" in col]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

# Creating the Customer Lifetime Value Data Structure

In [16]:
df.last_order_date.max()

Timestamp('2021-05-30 00:00:00')

In [18]:
df.last_order_date.min()

Timestamp('2020-05-30 00:00:00')

In [20]:
today_date=dt.datetime(2021,6,2)

### I want to create a new cltv dataframe to include customer_id, recency_cltv_weekly, T_weekly, frequency and monetary_cltv_avg values

In [21]:
df["total_price"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [22]:
df["first_last_order_difference"] = (df["last_order_date"]-df["first_order_date"]).dt.days

# Recency-Tenure-Frequency-Monetary

In [23]:
cltv_df=pd.DataFrame()

In [24]:
cltv_df["master_id"]=df["master_id"]

In [25]:
cltv_df["recency"]=df["first_last_order_difference"]

In [26]:
cltv_df["Tenure"]=(today_date-df["first_order_date"]).dt.days

In [27]:
cltv_df["frequency"]=df["omni_channel"]

In [28]:
cltv_df["monetary"]=df["total_price"]

In [29]:
cltv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   master_id  19945 non-null  object 
 1   recency    19945 non-null  int64  
 2   Tenure     19945 non-null  int64  
 3   frequency  19945 non-null  float64
 4   monetary   19945 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 779.2+ KB


In [31]:
cltv_df.sample(10)

,master_id,recency,Tenure,frequency,monetary
8962,7cbb1e82-a9e7-11e9-a2fc-000d3a38a36f,1794,1837,10.000,1728.900
17054,5b62d27a-2c1e-11ea-91ab-000d3a38a36f,593,606,5.000,1000.490
1927,bed40e92-9f19-11e9-9897-000d3a38a36f,823,860,3.000,580.070
7426,99182dc4-6cf4-11ea-b576-000d3a38a36f,434,482,7.000,1097.070
14122,bcc401de-73e8-11eb-b35d-000d3a38a36f,22,105,9.000,1928.780
14181,0e8c7b30-ac18-11e9-a2fc-000d3a38a36f,992,1119,48.000,8181.190
10197,a52ab84e-6d07-11ea-84a8-000d3a38a36f,294,482,7.000,838.360
9751,724cb758-63a0-11ea-a6dc-000d3a38a36f,352,449,3.000,937.740
12757,8f413578-567b-11ea-b9b9-000d3a38a36f,257,382,2.000,107.480
13620,51c0c5ac-b085-11e9-9757-000d3a38a36f,557,666,7.000,859.610


#  preparation recency-Tenure-Frequency-Monetary mectrics

In [32]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,19945.000,666.844,522.126,0.000,353.000,536.000,766.000,3034.000
Tenure,19945.000,802.303,523.397,6.000,518.000,652.000,837.000,3061.000
frequency,19945.000,4.978,4.116,2.000,3.000,4.000,6.000,52.000
monetary,19945.000,741.627,687.906,44.980,339.980,545.270,897.780,9063.760


In [33]:
cltv_df[cltv_df["frequency"]>1]

,master_id,recency,Tenure,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,119,215,5.000,939.370
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,1469,1575,21.000,2013.550
2,69b69676-1a40-11ea-941b-000d3a38a36f,366,553,5.000,585.320
3,1854e56c-491f-11eb-806e-000d3a38a36f,11,147,2.000,121.970
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,582,669,2.000,209.980
...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,288,620,3.000,401.960
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,296,458,2.000,390.470
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,621,630,3.000,632.940
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,689,798,6.000,1009.770


In [34]:
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

In [35]:
cltv_df["recency"]= cltv_df["recency"]/7

In [36]:
cltv_df["Tenure"] = cltv_df["Tenure"] / 7

# Establishment of BG-NBD Model

In [37]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [38]:
bgf.fit(cltv_df['frequency'],cltv_df['recency'],cltv_df['Tenure'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.43, b: 0.00, r: 3.67>

# Who are the customers that are expected the most purchase in 3 months?

In [39]:
cltv_df["expected_purc_3_months"] = bgf.conditional_expected_number_of_purchases_up_to_time(12,cltv_df['frequency'],cltv_df['recency'],cltv_df['Tenure'])

In [41]:
cltv_df["expected_purc_3_months"].sample(20)

18062   0.349
10328   0.726
5064    0.512
1558    0.487
6222    0.554
7186    0.561
1993    0.483
9605    0.246
16018   0.743
19633   0.826
4942    0.523
8753    0.318
16282   1.089
2772    0.524
13913   1.377
1005    0.219
17418   0.459
13679   0.531
12122   0.612
14006   0.439
Name: expected_purc_3_months, dtype: float64

# Who are the customers that are expected the most purchase in 6 months?

In [47]:
cltv_df["expected_purch_6_months"]=bgf.conditional_expected_number_of_purchases_up_to_time(24,cltv_df['frequency'],cltv_df['recency'],cltv_df['Tenure'])

In [48]:
cltv_df["expected_purch_6_months"].sort_values(ascending=False).head(10)

7330    9.287
15611   6.726
8328    6.263
19538   6.150
14373   5.984
10489   5.944
4315    5.650
6756    5.572
6666    5.538
10536   5.516
Name: expected_purch_6_months, dtype: float64

# Establishment of GAMMA-GAMMA Model

In [49]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

In [50]:
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

In [51]:
cltv_df["exp_average_profit"] = ggf.conditional_expected_average_profit(cltv_df['frequency'],cltv_df['monetary'])

# Calculation CLTV with BG-NBD model & GG model

In [52]:
cltv_df["cltv"] = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency'],
                                   cltv_df['Tenure'],
                                   cltv_df['monetary'],
                                   time=6,  # 6 Months
                                   freq="W",  # T's frequency info.
                                   discount_rate=0.01)

In [53]:
cltv_df.sort_values("cltv", ascending=False)

,master_id,recency,Tenure,frequency,monetary,expected_purc_3_months,expected_purch_6_months,exp_average_profit,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.857,8.000,4.000,1401.800,1.090,2.180,1449.061,3314.557
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.143,13.286,11.000,758.085,1.962,3.924,767.361,3159.549
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.714,101.143,7.000,1106.467,0.721,1.442,1127.612,1706.065
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.286,74.714,16.000,501.874,1.562,3.123,506.167,1658.716
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714,67.429,52.000,166.225,4.644,9.287,166.712,1624.547
...,...,...,...,...,...,...,...,...,...
2641,f7b3612e-a6ce-11e9-a2fc-000d3a38a36f,334.571,361.429,4.000,34.992,0.210,0.420,37.199,16.407
11232,f486e45e-a691-11e9-a2fc-000d3a38a36f,350.571,378.286,2.000,45.240,0.150,0.299,50.502,15.855
6963,7e0928c6-a6e3-11e9-a2fc-000d3a38a36f,342.000,352.857,4.000,32.935,0.214,0.429,35.074,15.778
27,c1f8f878-9f35-11e9-9897-000d3a38a36f,404.000,417.286,4.000,32.718,0.186,0.373,34.849,13.631


In [54]:
cltv_df["segment"]=pd.qcut(cltv_df["cltv"],4,labels=["D","C","B","A"])

In [55]:
cltv_df.groupby("segment").agg({"mean","sum"})

recency             Tenure         frequency          monetary  \
               sum    mean        sum    mean       sum  mean         sum   
segment                                                                     
D       692828.857 138.927 809165.286 162.255 18782.000 3.766  464521.610   
C       461307.714  92.521 562690.857 112.854 21924.000 4.397  627122.306   
B       409618.143  82.154 501735.571 100.629 25443.000 5.103  800898.116   
A       336275.714  67.444 412398.286  82.711 33131.000 6.645 1141093.900   

                expected_purc_3_months       expected_purch_6_months        \
           mean                    sum  mean                     sum  mean   
segment                                                                      
D        93.147               2036.623 0.408                4073.246 0.817   
C       125.777               2615.907 0.525                5231.814 1.049   
B       160.629               2992.196 0.600                5984.391 1.200   
A       228.860               3845.205 0.771                7690.411 1.542   

        exp_average_profit                cltv          
                       sum    mean         sum    mean  
segment                                                 
D               492157.738  98.688  400191.268  80.247  
C               659348.254 132.240  688574.745 138.102  
B               837587.883 167.988  993176.029 199.193  
A              1186940.350 238.055 1802665.093 361.545